In [1]:
import Pkg
pkgs = [
    "Eisenia",
    "Random",
    "Revise",
    "BioSequences",
    "Test",
    "LightGraphs",
    "Primes",
    "Statistics",
    "Plots",
    "uCSV",
    "DataFrames",
    "HTTP",
    "CodecZlib",
    "FASTX",
    "ProgressMeter"
]

for pkg in pkgs
    try
        Pkg.add(pkg)
    catch
#         # tried to install an unregistered local package
    end
    eval(Meta.parse("import $pkg"))
end

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


  Resolving package versions...
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes

In [2]:
TASK = "making-neo4j-graphs"
DATE = "2021-03-18"
DIR = "$(homedir())/$(DATE)-$(TASK)"
if !isdir(DIR)
    mkdir(DIR)
end
cd(DIR)

In [3]:
if !occursin("cypher-shell", ENV["PATH"])
    ENV["PATH"] = "/Users/cameronprybol/Software/cypher-shell:$(ENV["PATH"])"
end

"/Users/cameronprybol/Software/cypher-shell:/Applications/Julia-1.5.app/Contents/Resources/julia/bin:/Applications/Julia-1.5.app/Contents/Resources/julia/bin:/Users/cameronprybol/miniconda/bin:/Users/cameronprybol/.gem/ruby/2.6.0/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin"

In [4]:
function get_genome_from_ftp_directory(ftp_path)
    fasta_url = "$(ftp_path)/$(basename(ftp_path))_genomic.fna.gz"
    fasta_buffer = CodecZlib.GzipDecompressorStream(IOBuffer(HTTP.get(fasta_url).body))
    return collect(FASTX.FASTA.Reader(fasta_buffer))
end

get_genome_from_ftp_directory (generic function with 1 method)

In [18]:
function list_databases(;address="neo4j://localhost:7687", username="neo4j", password="password")
    io = open(`cypher-shell --address $(address) --username $(username) --password $(password) --database system --format auto "show databases"`)
    return DataFrames.DataFrame(uCSV.read(io, header=1, quotes='"', encodings=Dict("FALSE" => false, "TRUE" => true))...)
end

list_databases (generic function with 1 method)

In [19]:
function create_database(database_id; address="neo4j://localhost:7687", username="neo4j", password="password")
    current_databases = list_databases(;address, username, password)
    if database_id in current_databases[!, "name"]
        return true
    else
        run(`cypher-shell --address $(address) --username $(username) --password $(password) --database system --format auto "create database $(database_id)"`)
    end
end

create_database (generic function with 1 method)

In [20]:
function cypher(database_id, cmd; address="neo4j://localhost:7687", username="neo4j", password="password")
    run(`cypher-shell --address $(address) --username $(username) --password $(password) --database $(database_id) --format auto $(cmd)`)
end

cypher (generic function with 1 method)

In [8]:
db = "genbank"
assembly_summary_url = "https://ftp.ncbi.nlm.nih.gov/genomes/$(db)/assembly_summary_$(db).txt"
assembly_summary_file = "$(DIR)/$(basename(assembly_summary_url))"
if !isfile(assembly_summary_file)
    open(assembly_summary_file, "w") do io
        assembly_summary_response = HTTP.get(assembly_summary_url)
        write(io, assembly_summary_response.body)
    end
end
assembly_summary_table = DataFrames.DataFrame(uCSV.read(assembly_summary_file, delim='\t', header=2, skipmalformed=true)...)

,# assembly_accession,bioproject,biosample,wgs_master,refseq_category
,String,String,String,String,String
1,GCA_000001215.4,PRJNA13812,SAMN02803731,,reference genome
2,GCA_000001405.28,PRJNA31257,,,reference genome
3,GCA_000001515.5,PRJNA13184,SAMN02981217,AACZ00000000.4,na
4,GCA_000001545.3,PRJNA20869,SAMN02981238,ABGA00000000.1,na
5,GCA_000001635.9,PRJNA20689,,,reference genome
6,GCA_000001735.2,PRJNA10719,SAMN03081427,,reference genome
7,GCA_000001765.3,PRJNA10626,SAMN00779672,AADE00000000.2,na
8,GCA_000001895.4,PRJNA10629,SAMN02808228,AABR00000000.7,na
9,GCA_000001905.1,PRJNA12569,SAMN02953622,AAGU00000000.3,representative genome


In [9]:
genome_size_table_url = "ftp://ftp.ncbi.nlm.nih.gov/genomes/ASSEMBLY_REPORTS/species_genome_size.txt.gz"
genome_size_table_file = "$(DIR)/$(basename(genome_size_table_url))"
if !isfile(genome_size_table_file)
    open(genome_size_table_file, "w") do io
        genome_size_table_response = HTTP.get(genome_size_table_url)
        write(io, genome_size_table_response.body)
    end
end
genome_size_table_buffer = CodecZlib.GzipDecompressorStream(open(genome_size_table_file))
genome_size_table = DataFrames.DataFrame(uCSV.read(genome_size_table_buffer, header=1, delim='\t')...)

,#species_taxid,min_ungapped_length,max_ungapped_length,expected_ungapped_length
,Int64,Int64,Int64,Int64
1,9,318000,956000,636876
2,24,2950000,6050000,4500062
3,34,7405000,11109000,9257167
4,56,5994000,15000000,11989871
5,69,4891000,7337000,6113769
6,114,7212000,10820000,9015893
7,122,6222000,9334000,7777997
8,124,5461000,9315000,7387897
9,139,813000,1786000,1299537


In [10]:
joint_table = DataFrames.innerjoin(assembly_summary_table, genome_size_table, on=["species_taxid" => "#species_taxid"])
sort!(joint_table, "expected_ungapped_length")

,# assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid
,String,String,String,String,String,Int64
1,GCA_002829805.1,,,,na,230604
2,GCA_003972145.1,,,,na,230604
3,GCA_003972165.1,,,,na,230604
4,GCA_003972185.1,,,,na,230604
5,GCA_003972205.1,,,,na,230604
6,GCA_003972225.1,,,,na,230604
7,GCA_003972245.1,,,,na,230604
8,GCA_003972265.1,,,,na,230604
9,GCA_000851485.1,,,,reference genome,154834


In [11]:
# 112903 on refseq
# 795 on genbank!

In [12]:
group = first(DataFrames.groupby(joint_table, "taxid"))

,# assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid
,String,String,String,String,String,Int64
1,GCA_002829805.1,,,,na,230604
2,GCA_003972145.1,,,,na,230604
3,GCA_003972165.1,,,,na,230604
4,GCA_003972185.1,,,,na,230604
5,GCA_003972205.1,,,,na,230604
6,GCA_003972225.1,,,,na,230604
7,GCA_003972245.1,,,,na,230604
8,GCA_003972265.1,,,,na,230604


In [13]:
# how many groups are there?

In [14]:
show(group, allcols=true)

8×26 SubDataFrame
│ Row │ # assembly_accession │ bioproject │ biosample │ wgs_master │
│     │ String               │ String     │ String    │ String     │
├─────┼──────────────────────┼────────────┼───────────┼────────────┤
│ 1   │ GCA_002829805.1      │            │           │            │
│ 2   │ GCA_003972145.1      │            │           │            │
│ 3   │ GCA_003972165.1      │            │           │            │
│ 4   │ GCA_003972185.1      │            │           │            │
│ 5   │ GCA_003972205.1      │            │           │            │
│ 6   │ GCA_003972225.1      │            │           │            │
│ 7   │ GCA_003972245.1      │            │           │            │
│ 8   │ GCA_003972265.1      │            │           │            │

│ Row │ refseq_category │ taxid  │ species_taxid │ organism_name     │
│     │ String          │ Int64  │ Int64         │ String            │
├─────┼─────────────────┼────────┼───────────────┼───────────────────┤
│ 1   │ n

In [15]:
taxid = group[1, "taxid"]

230604

In [16]:
database_id = "taxid$(taxid)"

"taxid230604"

In [21]:
# delete the database if it exists
cypher("system", "DROP DATABASE $(database_id)")
# build it again
create_database(database_id)

Process(`cypher-shell --address neo4j://localhost:7687 --username neo4j --password password --database system --format auto 'create database taxid230604'`, ProcessExited(0))

In [22]:
ftp_path = group[1, "ftp_path"]

"ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/002/829/805/GCA_002829805.1_ASM282980v1"

In [23]:
accession = group[1, "# assembly_accession"]

"GCA_002829805.1"

In [24]:
for (accession, ftp_path) in DataFrames.eachrow(group[!, ["# assembly_accession", "ftp_path"]])
    for fasta in get_genome_from_ftp_directory(ftp_path)
        identifier = FASTX.identifier(fasta)
        description = FASTX.description(fasta)
        sequence = FASTX.sequence(fasta)
        cypher_call = "MERGE (g:GENOME {accession: '$(accession)'})<-[s:SOURCE]-(f:FASTA {identifier: '$(identifier)', description: '$(description)', sequence: '$(sequence)'}) RETURN *"
        cypher(database_id, cypher_call)
    end
end

f, g, s
(:FASTA {identifier: "AY247793.1", sequence: "CCACGTCTAGCTGCTGACGTTAAATGCAGCCGGGTGGTAATGCGTATTAATATTTCAACAACATGAATAAGCGCATTAATAATAATCGGAGAACCATGAGATCACGTAGGGGACGCGGTAGGACAATGGGATCTAATCTCATTCCTTACGCCAATTCACCAGTCCCTATACCATATACACCACCCGTTACCCCAGTCACCGTCATTAGTAATCCTCGGAAAACTACTTGGATTGACATTGATCTTTCAAGTGAAGAGTCCGGGATTTACACATTGGCGGTTGGGTCATACCGTAATAGGATTACTAAACTTGGTCCATCTAAACCTAACTTTATTATTGAGAAGGTTGCAGCATACGCTGCACCAGGAGATTATAAGGTTGTTCTCAATGACTTTAAAACTGGCATACAAGTCGTTGATGAAGGCTCTTATGCTCATAGAGCCGCAGTAGGTATTCTTTATCCACCTGCTGCACAAATATTTTACGGTATTTCAGCAACAGGCACACTCAATACTATCACTACCACTGCTAAAGATCCAGTTCCAGTGGTTCGTGCCTTGGTAACATATTGGGACTCCGAACAGTAATGAGCAGGTTCTATGCTTCGAACTAAAACAAGCAATGAGTTCTAGCTCCGAACTAAAACGAGCCAAACTCTCGCATCTGAGTATAAATCATGCCCCATGATCCTCGCATCGTATCGCACTTTGCTGGAGAAAACCCTTATGGCCAATTGTGGCAAGGGGAGACGTATTGGTTAGGCATTATTCCGGGTGCGTCGATAAATAAAGACCTT", description: "Macrobrachium rosenbergii XSV virus capsid protein gene, complete cds"}), (:GENOME {accession: "GCA_002829805.1"}), [:SOURCE]
f, g, s

In [ ]:
for

In [ ]:
# graphs = []
# for record in get_genome_from_ftp_directory(group[1, "ftp_path"])
#     graph = Eisenia.KmerGraph(BioSequences.DNAMer{7}, record)
#     push!(graphs, graph)
# end
# graphs

In [52]:
genome_accession = group[1, "# assembly_accession"]
genome = get_genome_from_ftp_directory(group[1, "ftp_path"])

1-element Array{FASTX.FASTA.Record,1}:
 FASTX.FASTA.Record:
   identifier: AY247793.1
  description: Macrobrachium rosenbergii XSV virus capsid protein gene, complete cds
     sequence: CCACGTCTAGCTGCTGACGTTAAATGCAGCCGGGTGGTA…

In [38]:
k = 7
node_type = BioSequences.DNAMer{k}

BioSequences.Mer{BioSequences.DNAAlphabet{2},7}

In [40]:
edge_type = typeof(
    (src = (node = node_type(BioSequences.randdnaseq(k)), orientation = true),
    dst = (node = node_type(BioSequences.randdnaseq(k)), orientation = true)))

NamedTuple{(:src, :dst),Tuple{NamedTuple{(:node, :orientation),Tuple{BioSequences.Mer{BioSequences.DNAAlphabet{2},7},Bool}},NamedTuple{(:node, :orientation),Tuple{BioSequences.Mer{BioSequences.DNAAlphabet{2},7},Bool}}}}

In [41]:
# edge_type = Pair{Pair{node_type, Bool}, Pair{node_type, Bool}}
evidence_type = typeof((accession = "", identifier = "", index = 1, orientation = true))

NamedTuple{(:accession, :identifier, :index, :orientation),Tuple{String,String,Int64,Bool}}

In [54]:
nodes = Dict{node_type, Set{evidence_type}}()

Dict{BioSequences.Mer{BioSequences.DNAAlphabet{2},7},Set{NamedTuple{(:accession, :identifier, :index, :orientation),Tuple{String,String,Int64,Bool}}}}()

In [55]:
edges = Dict{edge_type, Set{evidence_type}}()

Dict{NamedTuple{(:src, :dst),Tuple{NamedTuple{(:node, :orientation),Tuple{BioSequences.Mer{BioSequences.DNAAlphabet{2},7},Bool}},NamedTuple{(:node, :orientation),Tuple{BioSequences.Mer{BioSequences.DNAAlphabet{2},7},Bool}}}},Set{NamedTuple{(:accession, :identifier, :index, :orientation),Tuple{String,String,Int64,Bool}}}}()

In [56]:
graph = Dict(:nodes => nodes, :edges => edges)

Dict{Symbol,Dict{K,Set{NamedTuple{(:accession, :identifier, :index, :orientation),Tuple{String,String,Int64,Bool}}}} where K} with 2 entries:
  :nodes => Dict{BioSequences.Mer{BioSequences.DNAAlphabet{2},7},Set{NamedTuple…
  :edges => Dict{NamedTuple{(:src, :dst),Tuple{NamedTuple{(:node, :orientation)…

In [57]:
record = first(genome)
# for record in genome
display(record)
n = k+1
# for i in 1:1
i = 1
#     for i in 1:length(FASTX.sequence(record))-n+1
slice = view(FASTX.sequence(record),i:i+n-1)
display(slice)

raw_src = node_type(slice[1:end-1])
canonical_src = BioSequences.canonical(raw_src)
src_orientation = canonical_src == raw_src
src_evidence = (
    accession = genome_accession,
    identifier = FASTX.identifier(record),
    index = i,
    orientation = src_orientation)
nodes[canonical_src] = push!(get(nodes, canonical_src, Set{evidence_type}()), src_evidence)

raw_dst = node_type(slice[2:end])
canonical_dst = BioSequences.canonical(raw_dst)
dst_orientation = canonical_dst == raw_dst
dst_evidence = (
    accession = genome_accession,
    identifier = FASTX.identifier(record),
    index = i+1,
    orientation = dst_orientation)
nodes[canonical_dst] = push!(get(nodes, canonical_dst, Set{evidence_type}()), dst_evidence)

#     end
# end

FASTX.FASTA.Record:
   identifier: AY247793.1
  description: Macrobrachium rosenbergii XSV virus capsid protein gene, complete cds
     sequence: CCACGTCTAGCTGCTGACGTTAAATGCAGCCGGGTGGTA…

8nt DNA Sequence:
CCACGTCT

Set{NamedTuple{(:accession, :identifier, :index, :orientation),Tuple{String,String,Int64,Bool}}} with 1 element:
  (accession = "GCA_002829805.1", identifier = "AY247793.1", index = 2, orienta…

In [ ]:
edge_type = typeof(
    (src = (node = node_type(BioSequences.randdnaseq(k)), orientation = true),
    dst = (node = node_type(BioSequences.randdnaseq(k)), orientation = true)))

In [58]:
edge = (src = (node = canonical_src, orientation = src_orientation),
        dst = (node = canonical_dst, orientation = dst_orientation))

(src = (node = CCACGTC, orientation = true), dst = (node = AGACGTG, orientation = false))

In [63]:
function canonical(edge)
    if edge.src.node < edge.dst.node
        return edge
    else
        new_src = edge.dst.node
        @assert new_src == BioSequences.canonical(new_src)
        new_dst = edge.src.node
        @assert new_dst == BioSequences.canonical(new_dst)
        new_src_orientation = !edge.dst.orientation
        new_dst_orientaion = edge.dst.orientation ? !dge.src.orientation : !edge.src.orientation
        new_edge = (src = (node = new_src, orientation = new_src_orientation),
                    dst = (node = new_dst, orientation = new_dst_orientaion))
        return new_edge
    end
end

canonical (generic function with 1 method)

In [65]:
canonical_edge = canonical(edge)

(src = (node = AGACGTG, orientation = true), dst = (node = CCACGTC, orientation = false))

In [67]:
src_node = canonical_edge.src.orientation ? canonical_edge.src.node : BioSequences.reverse_complement(canonical_edge.src.node)

DNA 7-mer:
AGACGTG

In [68]:
dst_node = canonical_edge.dst.orientation ? canonical_edge.dst.node : BioSequences.reverse_complement(canonical_edge.dst.node)

DNA 7-mer:
GACGTGG

In [69]:
@assert BioSequences.LongSequence(src_node)[2:end] == BioSequences.LongSequence(dst_node)[1:end-1]

In [71]:
seq = push!(BioSequences.LongSequence(src_node), last(dst_node))

8nt DNA Sequence:
AGACGTGG

In [74]:
slice == seq || slice == BioSequences.reverse_complement(seq)

true

In [ ]:
graph = first(graphs)

In [ ]:
cypher_command = join(["MERGE (:DNAMer {sequence: \"$(kmer)\"})" for kmer in graph.kmers], '\n')
cypher(database_id, cypher_command)

In [ ]:
# edge = {orientation: [true, true], evidence: ["source-identifier_sequence-identifier_index_orientation"]}
# node = {sequence: "", evidence: ["source-identifier_sequence-identifier_index_orientation"]}

In [ ]:
# cyper_commands = []
ProgressMeter.@showprogress for edge_evidence in collect(graph.edge_evidence)
    edge = first(edge_evidence)
    evidence = last(edge_evidence)
    src_kmer = graph.kmers[edge.src]
    dst_kmer = graph.kmers[edge.dst]
    possible_orientations = map(bool -> Eisenia.assess_path_orientations([edge.src, edge.dst], graph.kmers, bool), [true, false])
    possible_orientations = filter!(x -> x != nothing, possible_orientations)
    orientations = first(possible_orientations)
    string_orienations = string(Int.(orientations))
    cypher_command = 
    """
    MATCH (src:DNAMer {sequence: "$(src_kmer)"})
    MATCH (dst:DNAMer {sequence: "$(dst_kmer)"})
    MERGE (src)-[edge:EDGE {orientations: $(string_orienations)}]->(dst)
    RETURN *
    """
#     push!(cyper_commands, cypher_command)
    cypher(database_id, cypher_command)
end

In [ ]:
# cyper_commands = []
ProgressMeter.@showprogress for edge_evidence in collect(graph.edge_evidence)
    edge = first(edge_evidence)
    evidence = last(edge_evidence)
    src_kmer = graph.kmers[edge.src]
    dst_kmer = graph.kmers[edge.dst]
    unique_sources = unique(map(e -> e.record_identifier, evidence))
    for unique_source in unique_sources
        cypher_command = 
        """
        MATCH (src:DNAMer {sequence: "$(src_kmer)"})
        MATCH (dst:DNAMer {sequence: "$(dst_kmer)"})
        MATCH (f:FASTA) WHERE f.identifier = "$(unique_source)"
        MERGE (src)-[src_f:CONTAINED_IN]->(f)
        MERGE (dst)-[dst_f:CONTAINED_IN]->(f)
        RETURN *
        """
        cypher(database_id, cypher_command)
    end
end